In [2]:
#experimenting with torch first


import torch
import torch.nn as nn

class CNNClassifier(nn.Module):

  def __init__(self, in_channels, n_classes):
    super().__init__()
    self.conv_block1 = nn.Sequential(
        nn.Conv2d(in_channels, 32, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(32),
        nn.ReLU()
    )

    self.conv_block2 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size = 1, stride = 1, padding = 1),
        nn.BatchNorm2d(32),
        nn.ReLU()
    )

    self.decoder = nn.Sequential(
        nn.Linear(32 * 28 * 28, 1024),
        nn.Sigmoid(),
        nn.Linear(1024, n_classes)
    )


  def forward(self, x):
    x = self.conv_block1(x)
    x = self.conv_block2(x)
    x = x.view(x.size(0), -1)

    x = self.decoder(x)
    return x  

In [3]:
model = CNNClassifier(1, 10)
print(model)

CNNClassifier(
  (conv_block1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv_block2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (decoder): Sequential(
    (0): Linear(in_features=25088, out_features=1024, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=1024, out_features=10, bias=True)
  )
)


In [4]:
# function for the conv block

def conv_block(in_c, out_c, *args, **kwargs):
  return nn.Sequential(
      nn.Conv2d(in_c, out_c, *args, **kwargs),
      nn.BatchNorm2d(out_c),
      nn.ReLU()
  )


class New_CNNClassifier(nn.Module):

  def __init__(self, in_channels, n_classes):
    super().__init__()
    self.conv_block1 = conv_block(in_channels, 32, kernel_size = 3, padding = 1)

    self.conv_block2 = conv_block(32, 64, kernel_size = 3, padding = 1)

    self.decoder = nn.Sequential(
        nn.Linear(32 * 28 * 28, 1024),
        nn.Sigmoid(),
        nn.Linear(1024, n_classes)
    )


  def forward(self, x):
    x = self.conv_block1(x)
    x = self.conv_block2(x)
    x = x.view(x.size(0), -1)

    x = self.decoder(x)
    return x  

model = New_CNNClassifier(1, 10)
print(model)

New_CNNClassifier(
  (conv_block1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv_block2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (decoder): Sequential(
    (0): Linear(in_features=25088, out_features=1024, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=1024, out_features=10, bias=True)
  )
)


In [6]:
#even more compaction in the layers

class New_CNNClassifier(nn.Module):

  def __init__(self, in_channels, n_classes):
    super().__init__()
    self.encoder = nn.Sequential(
        conv_block(in_channels, 32, kernel_size = 3, padding = 1),
        conv_block(32, 64, kernel_size = 3, padding = 1)
    )

    self.decoder = nn.Sequential(
        nn.Linear(32 * 28 * 28, 1024),
        nn.Sigmoid(),
        nn.Linear(1024, n_classes)
    )


  def forward(self, x):
    x = self.encoder(x)
    
    x = x.view(x.size(0), -1)

    x = self.decoder(x)
    return x  

model = New_CNNClassifier(1, 10)
print(model)

New_CNNClassifier(
  (encoder): Sequential(
    (0): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
  (decoder): Sequential(
    (0): Linear(in_features=25088, out_features=1024, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=1024, out_features=10, bias=True)
  )
)


In [8]:
### If we want to add new layers in the network --- such as

###############

encoder = nn.Sequential(
    conv_block(1, 32, kernel_size = 3, padding = 1),
    conv_block(32, 64, kernel_size = 3, padding = 1),
    conv_block(64, 128, kernel_size = 3, padding = 1),
    conv_block(128, 256, kernel_size = 3, padding = 1)
)

###############

In [11]:
#passing the layers as a list to sequential operator

class ListCNN(nn.Module):
  def __init__(self, in_c, n_classes):
    super().__init__()
    self.sizes = [in_c, 32, 64]

    conv_blocks = [conv_block(in_c, out_c, kernel_size = 3, padding = 1)
                  for in_c, out_c in zip(self.sizes, self.sizes[1:])]

    self.encoder = nn.Sequential(*conv_blocks)
    self.decoder = nn.Sequential(
        nn.Linear(32 * 28 * 28, 1024),
        nn.Sigmoid(),
        nn.Linear(1024, n_classes)
    )              


  def forward(self, x):
    x = self.encoder(x)
    x = x.view(x.size(0), -1)  
    x = self.decoder(x)


    return x


model = ListCNN(1, 10)
print(model)    

ListCNN(
  (encoder): Sequential(
    (0): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
  (decoder): Sequential(
    (0): Linear(in_features=25088, out_features=1024, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=1024, out_features=10, bias=True)
  )
)


In [12]:
#### this is easy to add a new layer if we need to 

# pass in a new size in the sizes argument

class ListCNN(nn.Module):
  def __init__(self, in_c, n_classes, sizes):
    super().__init__()
    self.e_sizes = [in_c, *sizes]

    conv_blocks = [conv_block(in_c, out_c, kernel_size = 3, padding = 1)
                  for in_c, out_c in zip(self.e_sizes, self.e_sizes[1:])]

    self.encoder = nn.Sequential(*conv_blocks)
    self.decoder = nn.Sequential(
        nn.Linear(32 * 28 * 28, 1024),
        nn.Sigmoid(),
        nn.Linear(1024, n_classes)
    )              


  def forward(self, x):
    x = self.encoder(x)
    x = x.view(x.size(0), -1)  
    x = self.decoder(x)


    return x


model = ListCNN(1, 10, [32, 64, 128])
print(model) 

ListCNN(
  (encoder): Sequential(
    (0): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
  (decoder): Sequential(
    (0): Linear(in_features=25088, out_features=1024, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=1024, out_features=10, bias=True)
  )
)


In [15]:
### making the decoder also into a block

def decoder_block(in_c, out_c):
  return nn.Sequential(
      nn.Linear(in_c, out_c),
      nn.Sigmoid()
  )


class NewListCNN(nn.Module):
  def __init__(self, in_c, enc_sizes, dec_sizes, n_classes):
    super().__init__()
    self.enc_sizes = [in_c, *enc_sizes] 
    self.dec_sizes = [32*28*28, *dec_sizes]

    conv_blocks = [conv_block(in_c, out_c, kernel_size = 3, padding = 1)
                  for in_c, out_c in zip(self.enc_sizes, self.enc_sizes[1:])]

    self.encoder = nn.Sequential(*conv_blocks)

    dec_blocks = [decoder_block(in_c, out_c)
                  for in_c, out_c in zip(self.dec_sizes, self.dec_sizes[1:])]

    self.decoder = nn.Sequential(*dec_blocks)

    self.final = nn.Linear(self.dec_sizes[-1], n_classes)


  def forward(self, x):

    x = self.encoder(x)
    x = x.view(x.size(0), -1)
    x = self.decoder(x)

    return x                  

In [16]:
model = NewListCNN(1, [32, 64], [1024, 512], 10)
print(model)

NewListCNN(
  (encoder): Sequential(
    (0): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
  (decoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=25088, out_features=1024, bias=True)
      (1): Sigmoid()
    )
    (1): Sequential(
      (0): Linear(in_features=1024, out_features=512, bias=True)
      (1): Sigmoid()
    )
  )
  (final): Linear(in_features=512, out_features=10, bias=True)
)


In [27]:

class MyEncoder(nn.Module):
    def __init__(self, enc_sizes):
        super().__init__()
        self.conv_blocks = nn.Sequential(*[conv_block(in_f, out_f, kernel_size=3, padding=1) 
                       for in_f, out_f in zip(enc_sizes, enc_sizes[1:])])

        def forward(self, x):
            return self.conv_blocks(x)
        
class MyDecoder(nn.Module):
    def __init__(self, dec_sizes, n_classes):
        super().__init__()
        self.dec_blocks = nn.Sequential(*[decoder_block(in_f, out_f) 
                       for in_f, out_f in zip(dec_sizes, dec_sizes[1:])])
        self.last = nn.Linear(dec_sizes[-1], n_classes)

    def forward(self, x):
        return self.dec_blocks(x)
    
    
class MyCNNClassifier(nn.Module):
    def __init__(self, in_c, enc_sizes, dec_sizes,  n_classes):
        super().__init__()
        self.enc_sizes = [in_c, *enc_sizes]
        self.dec_sizes = [32 * 28 * 28, *dec_sizes]

        self.encoder = MyEncoder(self.enc_sizes)
        
        self.decoder = MyDecoder(self.dec_sizes, n_classes)
        
    def forward(self, x):
        x = self.encoder(x)
        
        x = x.flatten(x.size(0), -1) # flat
        
        x = self.decoder(x)
        
        return x

In [28]:
model = MyCNNClassifier(1, [32,64], [1024, 512], 10)
print(model)

MyCNNClassifier(
  (encoder): MyEncoder(
    (conv_blocks): Sequential(
      (0): Sequential(
        (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (1): Sequential(
        (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
    )
  )
  (decoder): MyDecoder(
    (dec_blocks): Sequential(
      (0): Sequential(
        (0): Linear(in_features=25088, out_features=1024, bias=True)
        (1): Sigmoid()
      )
      (1): Sequential(
        (0): Linear(in_features=1024, out_features=512, bias=True)
        (1): Sigmoid()
      )
    )
    (last): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [30]:
class MyModule(nn.Module):
  def __init__(self, sizes):
    super().__init__()
    self.layers = nn.ModuleList([nn.Linear(in_c, out_c)
                                  for in_c, out_c in zip(sizes, sizes[1:])])
    
    self.cache = []


  def forward(self, x):
   for layer in self.layers:
     x = layer(x)
     self.cache.append(x)

   return x     

In [31]:
model = MyModule([1, 16, 32])
inputs = torch.rand((4, 1))
out = model(inputs)

[print(cache.shape) for cache in model.cache]

torch.Size([4, 16])
torch.Size([4, 32])


[None, None]

In [ ]:
def conv_block(in_c, out_c, activation = 'relu', *args, **kwargs):

  activations = nn.ModuleDict([
                               ['lrelu', nn.LeakyReLU()],
                               ['relu', nn.ReLU()]
  ])

  return nn.Sequential(
      nn.Conv2d(in_c, out_c, *args, **kwargs),
      nn.BatchNorm2d(out_c),
      activations[activation]
  )

  model1 = conv_block(1, 32, 'lrelu', kernel_size = 3, padding = 1)
  model2 = conv_block(1, 32, 'relu', kernel_size = 3, padding = 1)

  print(model1, model2)